In [ ]:
import os
import json
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
df_list = []
for root, dirs, files in os.walk('../results'):
    if 'config.json' in files:
        # print(files)
        config = json.load(open(os.path.join(root, 'config.json')))
        results_dev = json.load(open(os.path.join(root, 'results_dev.json')))
        results_test = json.load(open(os.path.join(root, 'results_test.json')))
        model_name = config['model_name'].split('/')[-1]
        best_epoch_dev = results_dev[np.argmax([el['micro_f1'] for el in results_dev])]['epoch']
        entry = {
            'model_name': model_name,
            'seed': config['seed'],
            'tags': config['use_prompt_tags'],
            'n_icl': config['n_icl_samples'],
            'epoch': best_epoch_dev,
            'micro_f1_dev': results_dev[best_epoch_dev]['micro_f1'],
            'micro_f1': results_test['micro_f1'],
            'micro_p': results_test['micro_precision'],
            'micro_r': results_test['micro_recall'],
        }
        metric_dict = {k: v['f1'] for k, v in results_test['per_tag_metrics'].items()}
        entry.update(metric_dict)
        # print(entry)
        df_list.append(entry)
df = pd.DataFrame(df_list)
df = df.sort_values('micro_f1', ascending=False)
df.round(3)
for col in df.columns:
    if is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(0)
        if (df[col] == 0).all():
            df = df.drop(col, axis=1)
# col_set = set(df.columns)
# col_set.remove('seed')
df = df.drop(['model_name',], axis = 1)
df['epoch'] = df['epoch'].apply(lambda x: x+1)
df_group = (
    df.drop(columns=['seed'])  # marginalize over seed
      .groupby(['tags', 'n_icl'])
      .agg(['mean', 'std'])  # compute both mean and std
      .round(3)
)
# df_group.columns = ['_'.join(col) for col in df_group.columns]  # flatten MultiIndex
df_group = df_group.reset_index()
# df_group.insert(loc=0, column='model', value=model_name)
df_group = df_group[df_group.columns[:12]]
latex_filename = '../paper/results_micro.tex'
os.makedirs(os.path.dirname(latex_filename), exist_ok=1)
df_group.to_latex(latex_filename, label='results_micro', caption=f'Micro-averaged results for {model_name}.', float_format="%.3f", index=False, escape=True)
df_group

tags n_icl epoch        micro_f1_dev        micro_f1        micro_p         \
              mean    std         mean    std     mean    std    mean    std   
0    0     0   3.0  0.000        0.366  0.027    0.377  0.043   0.423  0.052   
1    0     5   2.0  0.707        0.389  0.027    0.392  0.041   0.412  0.038   
2    0    10   1.8  0.447        0.406  0.027    0.389  0.068   0.413  0.094   
3    0    15   1.0  0.000        0.277  0.022    0.279  0.034   0.283  0.027   
4    0    20   1.0  0.000        0.279  0.077    0.269  0.042   0.265  0.034   
5    1     0   3.0  0.000        0.338  0.023    0.336  0.053   0.385  0.061   
6    1     5   2.2  0.837        0.218  0.014    0.204  0.039   0.163  0.031   
7    1    10   2.6  0.894        0.213  0.023    0.209  0.040   0.170  0.036   
8    1    15   2.4  0.894        0.217  0.022    0.217  0.046   0.172  0.046   
9    1    20   2.6  0.894        0.210  0.029    0.203  0.040   0.163  0.031   

  micro_r         
     mean    std  
0   0.340  0.037  
1   0.374  0.046  
2   0.368  0.050  
3   0.277  0.047  
4   0.279  0.063  
5   0.297  0.047  
6   0.275  0.058  
7   0.278  0.053  
8   0.299  0.051  
9   0.273  0.057

In [163]:
import sys
import re
sys.path.append('..')
from src.data_utils import convert_files
df = convert_files('../data')
df['tags'] = df.apply(lambda x: [el[0] for el in re.findall(r'<([A-Z]+[A-Z0-9]*)\s+[^>]*corr="[^"]*"[^>]*>(.*?)</\1>', x['text_an'])], axis = 1)

def flatten_deep(lst):
    return [item for sublist in lst 
            for item in (flatten_deep(sublist) if isinstance(sublist, list) else [sublist])]
tags_list = flatten_deep(df['tags'])
tags_list = pd.Series(tags_list)
counts = tags_list.value_counts()
sum((counts.values < 10).astype(int))
# sum((tags_list > 9).int())

np.int64(22)